In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, sampler
import torchvision.transforms as T
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

torch.cuda.is_available()

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
device = torch.device('cuda')
dtype = torch.float32

class digit_loader(Dataset):
    
    def __init__(self, train=False):
        if train == True:
            xy = pd.read_csv('../input/train.csv')
            xy = xy.values
            data = xy[:, 1:]
            labels = xy[:,0]
            self.labels = torch.tensor(labels, device=device)
        else:
            xy = pd.read_csv('../input/test.csv')
            data = xy.values
        N, D = data.shape
        H = W = int(np.sqrt(D))
        self.data = torch.tensor(data.reshape(N, 1, H, W), dtype=dtype, device=device)
        self.len = N
        
    def __getitem__(self, index):
        try:
            return self.data[index], self.labels[index]
        except:
            return self.data[index]
    
    def __len__(self):
        return self.len
       
BATCH = 32
train_dataset = digit_loader(train=True)
N_train = int(train_dataset.len*0.8)
N_data = train_dataset.len
train_loder = DataLoader(train_dataset,
                        batch_size=BATCH,
                        sampler=sampler.SubsetRandomSampler(range(N_train)))
val_dataset = digit_loader(train=True)
val_loader = DataLoader(val_dataset,
                       batch_size=BATCH,
                       sampler = sampler.SubsetRandomSampler(range(N_train, N_data)))
test_dataset = digit_loader()
test_loader = DataLoader(test_dataset,
                        batch_size=32)

In [ ]:
import torch.nn.functional as F

def train(model, optimizer, epochs=25):
    model.to(device=device)
    for e in range(epochs):
        for t ,(batch_data, batch_labels) in enumerate(train_loder):
            model.train()
            scores = model(batch_data)
            loss = F.cross_entropy(scores, batch_labels)
            
            # Reinitialize parameters to 0
            optimizer.zero_grad()
            #back propagate to obtain all parameters' gradients
            loss.backward()
            # update parameters according to gradients
            optimizer.step()
            
            # monitor accuracy
            if t % 50 == 0:
                print ('epoch {0}, iteration {1}, loss = {2}'.format(e, t, loss.item()))
                check_accuracy(model, val_loader)
                print('cuda memory allocated %d' %torch.cuda.memory_allocated())
                print()
                
def check_accuracy(model, loader):
    model.eval()
    num_correct = 0
    num_samples = 0
    with torch.no_grad():
        for data, labels in loader:
            scores = model(data)
            _, predictions = scores.max(1)
            num_correct += (predictions == labels).sum()
            num_samples += labels.size(0)
    acc = float(num_correct)/num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100*acc))

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
in_channel = 1
channel_1 = 32
channel_2 = 16
channel_3 = 8
in_features = channel_3 * 28 * 28
num_classes = 10

model = None
optimizer = None

# 3 layer conv net
model = nn.Sequential(
                    nn.Conv2d(in_channel, channel_1, 3, padding=1, bias=True),
                    nn.ReLU(),
                    nn.Conv2d(channel_1, channel_2, 3, padding=1, bias=True),
                    nn.ReLU(),
                    nn.Conv2d(channel_2, channel_3, 3, padding=1, bias=True),
                    nn.ReLU(),
                    Flatten(),
                    nn.Linear(in_features, num_classes)
                    )

# optimizer stochastic gradient decent
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
train(model, optimizer, epochs=10)

In [ ]:
print(torch.cuda.memory_allocated())


In [ ]:
test = pd.read_csv('../input/test.csv')
test = test.values
N, D = test.shape
H = W = int(np.sqrt(D))
test_tensor = torch.tensor(test.reshape(N, 1, H, W), dtype=dtype, device=device)
test_scores = model(test_tensor)
_, predictions = test_scores.max(1)
prediction = predictions.cpu()
submission = pd.DataFrame.from_dict(dict([('ImageId', range(1,predictions.size(0)+1)), ('Label', predictions.cpu().numpy())]))
submission.set_index('ImageId', inplace=True)
submission.to_csv('submission.csv')